测试 HTTP 相关工具函数，例如 `ping()` 函数，或测试参数传递是否正确，通常有下列两种实现方案：
- 创建一个临时测试服务器
- 使用 Pytest 的 httpserver

In [1]:
!pip install pytest

### 使用 `http.server` 手写 TestHandler

In [4]:
%%writefile mymain.py

import requests

def call_api(url, payload, **kwargs):
    for k, v in payload.items():
        if v.startswith("$"):
            var_name = v[1:]
            v2 = kwargs.get(var_name)
            if v2:
                payload[k] = v2

    resp = requests.post(url, json=payload, timeout=5)
    return resp.json()

Writing mymain.py


In [9]:
%%writefile test_mymain.py

import json
import socket
import threading
from http.server import BaseHTTPRequestHandler, HTTPServer
import pytest

from mymain import call_api

class TestHandler(BaseHTTPRequestHandler):
    def do_POST(self):
        content_length = int(self.headers["Content-Length"])
        body = self.rfile.read(content_length)
        data = json.loads(body)

        assert data["name"] == "Tom"
        assert data["age"] == 18

        response = {"status": "ok"}
        self.send_response(200)
        self.send_header("Content-Type", "application/json")
        self.end_headers()
        self.wfile.write(json.dumps(response).encode())

    def log_message(self, format, *args):
        pass

@pytest.fixture
def http_server():
    sock = socket.socket()
    sock.bind(("", 0))
    port = sock.getsockname()[1]
    sock.close()

    server = HTTPServer(("localhost", port), TestHandler)

    thread = threading.Thread(target=server.serve_forever)
    thread.daemon = True
    thread.start()

    yield f"http://localhost:{port}"

    server.shutdown()
    thread.join()

def test_call_api(http_server):
    params = {"name": "Tom", "age": 18}
    call_api(url=http_server, payload={"name": "$name", "age": "$age"}, **params)

Overwriting test_mymain.py


In [10]:
!pytest -q

.                                                                        [100%]
=============================== warnings summary ===============================
test_mymain.py:10
  /content/test_mymain.py:10: PytestCollectionWarning: cannot collect test class 'TestHandler' because it has a __init__ constructor (from: test_mymain.py)
    class TestHandler(BaseHTTPRequestHandler):

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
1 passed, 1 warning in 0.52s


### PyTest TestServer

In [12]:
!pip install pytest-httpserver

In [17]:
%%writefile test_mymain2.py

from mymain import call_api

def test_call_api(httpserver):
    httpserver.expect_request(
        "/",
        method="POST",
        json={"name": "Tom", "age": 18},
    ).respond_with_json({"status": "ok"})

    params = {"name": "Tom", "age": 18}
    call_api(url=httpserver.url_for("/"), payload={"name": "$name", "age": "$age"}, **params)

Overwriting test_mymain2.py


In [18]:
!pytest -q

..                                                                       [100%]
=============================== warnings summary ===============================
test_mymain.py:10
  /content/test_mymain.py:10: PytestCollectionWarning: cannot collect test class 'TestHandler' because it has a __init__ constructor (from: test_mymain.py)
    class TestHandler(BaseHTTPRequestHandler):

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
2 passed, 1 warning in 0.53s
